# Laboratorio 2 - Parte 2

### Modelos no paramétricos

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer integrante
Nombre: Jorge Hiler Ricardo

#### Segundo integrante
Nombre: Santiago Gaviria Zapata

In [3]:
from __future__ import division
%matplotlib inline
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt


#Evitar algunas advertencias
import warnings
warnings.filterwarnings("always")

## Ejercicio 1: Contextualización del problema


Usaremos el dataset iris para el problema de clasificación, desde la librería sklearn. En el UCI Machine Learning Repository se encuentra más información en el siguiente [link](https://archive.ics.uci.edu/ml/datasets/iris) .

In [4]:
#cargamos la bd iris desde el dataset de sklearn
from sklearn import datasets

iris = datasets.load_iris()


X, Y = iris.data, iris.target

Responda las siguientes preguntas: 

1.1 ¿Cu&aacute;ntas clases tiene el problema?:


In [7]:
clases, cant = np.unique(Y, return_counts=True)
#print(clases, cant)
print("Número de clases: ", clases.size)

Número de clases:  3


1.2 ¿Cu&aacute;ntas caracter&iacute;sticas tiene el problema?:

In [10]:
m,c = X.shape
print("Número de caracteristicas: ", c)

Número de caracteristicas:  4


1.3 ¿Cu&aacute;ntas muestras tiene el problema?:

In [11]:
print("Número de muestras: ", m)

Número de muestras:  150


1.4 ¿Cu&aacute;ntas muestras por cada clase hay en la base de datos?:

In [16]:
z = zip(clases, cant)
print("Hay 50 muestras por cada clase")
print(set(z))

Hay 50 muestras por cada clase
{(1, 50), (0, 50), (2, 50)}


## Ejercicio 2: Completar código KNN

Analice los siguientes métodos de la teoría vista para los modelos de *K-vecinos más cercanos (KNN)* y complete el código del método KNN.

<b>Nota</b>: Para el cáculo de la distancia entre vectores tienen dos opciones, usar la función la distancia euclidiana `scipy.spatial.distance.euclidean`([Ejemplo](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html)) o usar la función `numpy.linalg.norm`([Ejemplo](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html)). Revisen la documentación para comprender su uso. También serán de utilidad las funciones `numpy.sort` y `numpy.argsort`.


<b>Nota</b>: Para el cáculo de la distancia entre vectores tienen dos opciones, usar la función scipy.spatial.distance.euclidean o usar la función numpy.linalg.norm. <b>Revisen la documentación </b> para comprender su uso. También serán de utilidad las funciones numpy.sort y numpy.argsort.

In [17]:
from scipy.stats import mode
import operator

from statistics import mean

#Para calcular el error en los problemas de clasificación
def ErrorClas(Y_lest, Y):
    error = 1 - np.sum(Y_lest == Y)/len(Y)
    
    return error


N = np.size(X,0)


def compareWithEntryValue(enteredValue, X_train):
    n = len(X_train)
    result = np.zeros(n)
    i = 0
    for xi in X_train:
        result[i] = sc.spatial.distance.euclidean(xi, enteredValue)
        i = i + 1
    return result
        

def nearestNeighbors(k, distances):
    nearests = distances[:k]
    return nearests

def sortAndGetIndexes(array):
    indexes = np.argsort(array)
    return indexes

def getYiValues(Y, nearest):
    n = len(nearest)
    yiArray = np.zeros(n)
    iter = 0
    for i in nearest:
        yiArray[iter] = Y[i]
        iter = iter + 1
    return yiArray

def KNN(X_train, Y_train, X_test, k, tipo):
    
    #X_train: Matriz con las muestras de entrenamiento
    #Y_train: Vector con los valores de salida pra cada una de las muestras de entrenamiento
            
    
    #X_val: Matriz con las muestras de validación
    #tipo: Bandera que indica si el problema es de regresión o de clasificación.
    
    #Parámetro k que equivale al número de vecinos a tener en cuenta para resolver el problema de 
    #predicción de la variable de salida
    k = k 
    Nt = len(X_test)
    Y_test = np.zeros(Nt)
    
    N = np.size(X_train, 0)
    
    i = 0
    for enteredValue in X_test:
        distances = compareWithEntryValue(enteredValue, X_train)
        orderedNearest = sortAndGetIndexes(distances)
        nearest = nearestNeighbors(k, orderedNearest)
        yiArray = getYiValues(Y_train, nearest)
        
        if(tipo == 1):
            selected = sc.stats.mode(yiArray)
            mode = selected[0]
            Y_test[i] = mode
        elif(tipo == 0):
            meanYi = mean(yiArray)
            Y_test[i] = meanYi
        i = i+1
    #Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_test, en el mismo orden.  
    return Y_test 

## Ejercicio 3

Una vez haya completado el codigo del método de KNN, ejecute varias veces el proceso de entrenamiento y evaluación cambiando el parametro $k$, el cual es el numero de vecinos, y complete la siguiente tabla con los valores del error de clasificación obtenidos:

In [27]:
from numpy import random
from numpy import matlib
from sklearn.model_selection import train_test_split
import math
N = np.size(X,0)

#Validamos el modelo
random.seed(1)
Error = np.zeros(4)
for j in range(4):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25)

    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = sc.stats.stats.zscore(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

    k = 5
    tipo = 1
    Yest = KNN(Xtrain, Ytrain, Xtest, k, tipo)#Complete con el llamado apropiado de la función que Uds diseñaron
    
    #Evaluamos las predicciones del modelo con los datos de test
    Error[j] = ErrorClas(Yest,Ytest)
print('\nError durante la prueba = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))


Error durante la prueba = 0.26973684210526316+-0.08802031684381349


Responda:

3.1 ¿Qué metodología de validación se usa en el experimento?


La metodología de validación usada en el experimento es BootStrap, donde se toma un conjunto de entrenamiento del 75% y un conjunto de prueba del 25% debido al número (pequeños) de muestras.

3.2 ¿Cuántas muestras se usan para el entrenamiento?


In [49]:
np.size(Xtrain, 0)

114

3.3 ¿Cuántas muestras se usan para la validación?

In [50]:
np.size(Xtest, 0)

36

#### Tabla de resultados

In [21]:
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7'])})
df_types["Error_Prueba"] = ""
df_types["Desviación estándar del error"] = ""
df_types.set_index(['Numero de vecinos'], inplace=True)
df_types["Error_Prueba"][1] = "0.2697"
df_types["Desviación estándar del error"][1] = "0.088"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [28]:
qgrid_widget.get_changed_df()

,Error_Prueba,Desviación estándar del error
Numero de vecinos,,
1,0.20394736842105263,0.11376063464335907
2,0.2697,0.088
3,0.26973684210526316,0.08603090020146066
4,0.28289473684210525,0.07529949435697103
5,0.26973684210526316,0.08802031684381349
6,0.2763157894736842,0.0842516347030638
7,0.2763157894736842,0.0842516347030638


## Ejercicio 4

A continuación debe completar el código del método parzen para resolver problemas de clasificación con el modelo ventana de Parzen.

In [29]:
#Calcula el kernel gaussiano de x
def kernel_gaussiano(x):
    return np.exp((-0.5)*x**2)

def ParzenWindow(x,Data,h):
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = sc.spatial.distance.euclidean(x,Data[k,:])
        suma += kernel_gaussiano(u/h)
    
    return suma

def splitClass(Y_train): #retorna los indices de las clases
    clases, cant = np.unique(Y_train, return_counts=True) #En cant está la cantidad de muestras de la clase
    n = len(clases)
    split_class = np.array([np.zeros(cant[0]), np.zeros(cant[1]), np.zeros(cant[2])]) #inicializa array de clases
    i = 0
    for j in clases:
        indices = np.where(Y_train == j)    #Todos los indices donde Y_train es igual a j
        indices = np.array(indices)[0]        
        split_class[i] = indices
        i= i + 1
    return split_class

#X_val vector de muestras de prueba 
def ParzenClass(X_train, Y_train, X_val, ancho_h):
    n = len(X_val)
    Yest = np.zeros(n)
    h = ancho_h
    clases, cant = np.unique(Y_train, return_counts=True) #En cant está la cantidad de muestras de la clase
    nc = len(clases)
    index_by_class = splitClass(Y_train)

    j = 0
    for x in X_val:
        fdp_mayor = 0
        for p in index_by_class:
            p = p.astype(int)
            clase = Y_train[p[0]] #Valor de la clase evaluada
            a = np.array(X_train) #x_train
            b = np.array(p) # indices_clase_p
            x_train_class_p = a[b] #obtener elementos en a en la posición b
            fdp_current = ParzenWindow(x, x_train_class_p, h)
            if fdp_current > fdp_mayor:
                Yest[j] = clase  
                fdp_mayor = fdp_current
        j = j + 1
        
    
    return Yest#Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.  

## Ejercicio 5

Ahora debe resolver el mismo problema de clasificación con el método ventana de Parzen. Complete el código ejecute las pruebas para diferentes valores de $h$ y llene la siguiente tabla:

In [39]:
from numpy import random
from sklearn.model_selection import StratifiedKFold
import math

#Validamos el modelo
random.seed(1)
Error = np.zeros(4)
skf = StratifiedKFold(n_splits=4)
j = 0
h=10000
for train, test in skf.split(X, Y): #separa las muestras
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = sc.stats.stats.zscore(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

    Yest = ParzenClass(Xtrain, Ytrain, Xtest, h) #Complete con el llamado apropiado de la función que Uds diseñaron
  
    #Evaluamos las predicciones del modelo con los datos de test
    Error[j] = ErrorClas(Yest,Ytest)
    j += 1
print('\nError durante la prueba = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))


Error durante la prueba = 0.2216880341880342+-0.04696907336459245


Responda:
    
5.1 ¿Qué metodología de validación se usó en la simulación?:

Validación cruzada 

5.2 ¿Cuántas muestras se usan para el entrenamiento?:



In [48]:
np.size(Xtrain,0)

114

5.3 ¿Cuántas muestras se usan para la validación?:



In [44]:
np.size(Xtest,0)

36

#### Tabla de resultados

In [32]:
import pandas as pd
import qgrid
randn = np.random.randn
df_types2 = pd.DataFrame({
    'Tamano de ventana' : pd.Series(['0.05', '0.1', '0.5', '1', '2', '5', '10'])})
df_types2["Error_Prueba"] = ""
df_types2["Desviación estándar del error"] = ""
df_types2.set_index(['Tamano de ventana'], inplace=True)
df_types2["Error_Prueba"][1] = "0.222"
df_types2["Desviación estándar del error"][1] = "0.043"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [41]:
qgrid_widget.get_changed_df()

,Error_Prueba,Desviación estándar del error
Tamano de ventana,,
0.05,0.2216880341880342,0.04332854434165311
0.1,0.222,0.043
0.5,0.281517094017094,0.04021339704891218
1,0.280982905982906,0.03145818774430656
2,0.24198717948717952,0.058466142137675314
5,0.2216880341880342,0.04696907336459245
10,0.2216880341880342,0.04696907336459245
